# 1- RETRIEVAL AND SEARCH 
# 2- USE LLM ALONE
# 3- MIX RAG WITH LLM 

In [ ]:
import minsearch as minsearch
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
import os

In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

<h3> 1- create our search engine consedering questions, text and section as information and the course as a key field</h3>

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [6]:
index.fit(documents)

In [7]:
boost = {'question': 3.0, 'section': 0.5} # Poids d'importance de chaque text field dans le moteur de recherche
q = 'the course has already started, can I still enroll?'
results = index.search(
        query=q,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )


In [8]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

<h3>After creat awer KB and our search engine, now is time to pass that to a LLM Model</h3>

In [ ]:
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

<h5>2- LLM Direct without search</h5>

In [12]:
response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": q}]
    )

In [13]:
response.choices[0].message.content

"You may still be able to enroll in a course that has already started, but it depends on the institution or platform offering the course. Some universities or online platforms have flexible enrollment policies and allow late enrollments, while others have strict deadlines. Here’s what you can do:\n\n1. **Check the Enrollment Policy**: Review the institution's website or contact the admissions office to learn about their policy on late enrollment.\n\n2. **Contact the Instructor**: Reach out to the course instructor to express your interest. They may allow late enrollment, especially if you have a valid reason.\n\n3. **Understand the Impact**: If you join late, you’ll need to catch up on missed work. Make sure you’re prepared for this.\n\n4. **Consider the Course Type**: Some courses, like self-paced online classes, might have more flexible deadlines compared to structured, in-person classes.\n\n5. **Ask About Prorating**: If the course has fees, inquire whether any fees will be prorated

<h5>3- LLM with our search engine and use it as the context of the LLM</h5>

In [14]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [15]:
#### BUILD CONTEXT
context = ""
    
for doc in results: # results of our engine
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [17]:
print(context)

section: General course-related questions
question: The course has already started. Can I still join it?
answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you c

In [19]:
#### Prompt is the imput of the LLM Model
prompt = prompt_template.format(question=q, context=context).strip()
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: the course has already started, can I still enroll?

CONTEXT: 
section: General course-related questions
question: The course has already started. Can I still join it?
answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final pr

In [20]:
response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

In [21]:
print(response.choices[0].message.content)

Yes, you can still enroll in the course even after it has started. You may not be able to submit some of the homework assignments, but you can participate in the course. To earn a certificate, you need to submit 2 out of 3 course projects and review 3 peers' projects by the deadline. Even if you join the course late, such as at the end of November, you can still be eligible for a certificate if you complete these requirements.
